In [2]:
%load_ext sql

In [3]:
%%sql

sqlite:///CourseDataWarehouseDemo.db
    
attach "CourseData.db" as db1

Done.


[]

In [3]:
%%sql

PRAGMA table_info(COURSE_MEETINGS);

 * sqlite:///CourseDataWarehouseDemo.db
Done.


cid,name,type,notnull,dflt_value,pk


Creating the tables for our Data Warehouse. There are only 5 tables, with one fact table.

In [16]:
%%sql

DROP TABLE IF EXISTS INSTRUCTOR_WH;
CREATE TABLE INSTRUCTOR_WH (
    IID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouseDemo.db
Done.
Done.


[]

In [6]:
%%sql

DROP TABLE IF EXISTS LOCATION_WH;
CREATE TABLE LOCATION_WH (
    LID INTEGER PRIMARY KEY,
    Location TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouseDemo.db
Done.
Done.


[]

In [7]:
%%sql

DROP TABLE IF EXISTS TIME_CODE_WH;
CREATE TABLE TIME_CODE_WH (
    TCID INT PRIMARY KEY,
    Meetings TEXT NULL
);

 * sqlite:///CourseDataWarehouseDemo.db
Done.
Done.


[]

In [8]:
%%sql

DROP TABLE IF EXISTS CATALOG_COURSE_WH;
CREATE TABLE CATALOG_COURSE_WH (
    CAID INTEGER PRIMARY KEY,
    CatalogID TEXT NOT NULL,
    Title TEXT NOT NULL,
    Attributes TEXT,
    Description TEXT,
    Prereqs TEXT
);

 * sqlite:///CourseDataWarehouseDemo.db
Done.
Done.


[]

In [9]:
%%sql

DROP TABLE IF EXISTS PROGRAM_WH;
CREATE TABLE PROGRAM_WH (
    PID INTEGER PRIMARY KEY,
    ProgramCode TEXT NOT NULL,
    ProgramName TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouseDemo.db
Done.
Done.


[]

In [10]:
%%sql

DROP TABLE IF EXISTS TERM_WH;
CREATE TABLE TERM_WH (
    TermID INTEGER PRIMARY KEY,
    Term TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouseDemo.db
Done.
Done.


[]

In [11]:
%%sql

DROP TABLE IF EXISTS REG_COURSE_WH;
CREATE TABLE REG_COURSE_WH (
    COID INTEGER PRIMARY KEY,
    TCID INTEGER,
    LID INTEGER,
    CAID INTEGER,
    IID INTEGER,
    TermID INTEGER,
    PID INTEGER, 
    Cap INTEGER,
    Act INTEGER,
    Rem INTEGER,
    Credits INTEGER,
    Day TEXT,
    Start INTEGER,
    End INTEGER, 
    FOREIGN KEY (TCID) REFERENCES TIME_CODE_WH(TCID),
    FOREIGN KEY (LID) REFERENCES LOCATION_WH(LID),
    FOREIGN KEY (CAID) REFERENCES CATALOG_COURSE_WH(TID),
    FOREIGN KEY (IID) REFERENCES INSTRUCTOR_WH(IID),
    FOREIGN KEY (TermID) REFERENCES TERM_WH(TermID),
    FOREIGN KEY (PID) REFERENCES PROGRAM_WH(PID)
    )

 * sqlite:///CourseDataWarehouseDemo.db
Done.
Done.


[]

Inserting data into the tables for the data warehouse to do some queries on and for user use.

In [30]:
%%sql

DELETE FROM INSTRUCTOR_WH;
INSERT INTO INSTRUCTOR_WH (Name)
    SELECT DISTINCT db1.INSTRUCTOR.Name
    FROM db1.INSTRUCTOR;
    
    SELECT Name
    FROM db1.OFFERINGS_COURSE

    LIMIT 10;


 * sqlite:///CourseDataWarehouseDemo.db
1104 rows affected.
1104 rows affected.
Done.


Name
C01
C02
C03
C04
C05
C06
G
H
I
J


In [17]:
%%sql

DELETE FROM LOCATION_WH;
INSERT INTO LOCATION_WH (LID, Location)
    SELECT DISTINCT db1.LOCATION.LID, db1.LOCATION.Location
    FROM db1.LOCATION;

 * sqlite:///CourseDataWarehouseDemo.db
0 rows affected.
207 rows affected.


[]

In [18]:
%%sql

DELETE FROM TIME_CODE_WH;
INSERT INTO TIME_CODE_WH (Meetings)
    SELECT DISTINCT OFFERINGS_COURSE.Meetings
    FROM db1.OFFERINGS_COURSE;
    

 * sqlite:///CourseDataWarehouseDemo.db
0 rows affected.
12039 rows affected.


[]

In [19]:
%%sql

DELETE FROM CATALOG_COURSE_WH;
INSERT INTO CATALOG_COURSE_WH (CatalogID, Title, Attributes, Description, Prereqs)
    SELECT DISTINCT CatalogID, Title, Attributes, Description, Prereqs
    FROM db1.CATALOG_COURSE;

 * sqlite:///CourseDataWarehouseDemo.db
0 rows affected.
2220 rows affected.


[]

In [20]:
%%sql

DELETE FROM PROGRAM_WH;
INSERT INTO PROGRAM_WH (ProgramCode, ProgramName)
    SELECT DISTINCT ProgramCode, ProgramName
    FROM db1.PROGRAM;

 * sqlite:///CourseDataWarehouseDemo.db
0 rows affected.
83 rows affected.


[]

In [21]:
%%sql

DELETE FROM TERM_WH;
INSERT INTO TERM_WH (Term)
    SELECT DISTINCT Term
    FROM db1.OFFERINGS_COURSE;

 * sqlite:///CourseDataWarehouseDemo.db
0 rows affected.
19 rows affected.


[]

In [12]:
%%sql

DELETE FROM REG_COURSE_WH;
INSERT INTO REG_COURSE_WH (TCID, LID, CAID, IID, TermID, PID, Cap, Act, Rem, Credits, Day, Start, End)
    SELECT DISTINCT db1.MEETINGS_COURSE.CMID, db1.LOCATION.LID, db1.CATALOG_COURSE.CAID,
                     INSTRUCTOR_WH.IID, TERM_WH.TermID, db1.PROGRAM.PID, db1.OFFERINGS_COURSE.Cap, 
                     db1.OFFERINGS_COURSE.Act, db1.OFFERINGS_COURSE.Rem, db1.CATALOG_COURSE.Credits, 
                     db1.MEETINGS_COURSE.Day, db1.MEETINGS_COURSE.Start, db1.MEETINGS_COURSE.End
FROM db1.OFFERINGS_COURSE
        LEFT JOIN db1.CATALOG_COURSE ON (OFFERINGS_COURSE.CatalogID = CATALOG_COURSE.CatalogID)
        LEFT JOIN db1.PROGRAM ON (CATALOG_COURSE.ProgramName = PROGRAM.ProgramName)
        LEFT JOIN db1.INSTRUCTOR ON (OFFERINGS_COURSE.Name = INSTRUCTOR.Name)
        LEFT JOIN db1.MEETINGS_COURSE ON (OFFERINGS_COURSE.CRN = MEETINGS_COURSE.CRN)
        LEFT JOIN db1.LOCATION ON (MEETINGS_COURSE.Location = LOCATION.Location)
        LEFT JOIN TERM_WH ON (OFFERINGS_COURSE.Term = TERM_WH.Term)
        LEFT JOIN INSTRUCTOR_WH ON (INSTRUCTOR.Name = INSTRUCTOR_WH.Name);
    

 * sqlite:///CourseDataWarehouseDemo.db
0 rows affected.
381524 rows affected.


[]

In [28]:
%%sql

SELECT DISTINCT db1.MEETINGS_COURSE.CMID, db1.LOCATION.LID, db1.CATALOG_COURSE.CAID,
                     INSTRUCTOR_WH.IID, TERM_WH.TermID, db1.PROGRAM.PID, db1.OFFERINGS_COURSE.Cap, 
                     db1.OFFERINGS_COURSE.Act, db1.OFFERINGS_COURSE.Rem, db1.CATALOG_COURSE.Credits, 
                     db1.MEETINGS_COURSE.Day, db1.MEETINGS_COURSE.Start, db1.MEETINGS_COURSE.End
FROM db1.OFFERINGS_COURSE
        LEFT JOIN db1.CATALOG_COURSE ON (OFFERINGS_COURSE.CatalogID = CATALOG_COURSE.CatalogID)
        LEFT JOIN db1.PROGRAM ON (CATALOG_COURSE.ProgramName = PROGRAM.ProgramName)
        LEFT JOIN INSTRUCTOR ON (OFFERINGS_COURSE.Name = INSTRUCTOR.Name)
        LEFT JOIN db1.MEETINGS_COURSE ON (OFFERINGS_COURSE.CRN = MEETINGS_COURSE.CRN)
        LEFT JOIN db1.LOCATION ON (MEETINGS_COURSE.Location = LOCATION.Location)
        LEFT JOIN TERM_WH ON (OFFERINGS_COURSE.Term = TERM_WH.Term)
        LEFT JOIN INSTRUCTOR_WH ON (INSTRUCTOR.IID = INSTRUCTOR_WH.IID)
WHERE LOCATION.LID = 10
    
LIMIT 20;


 * sqlite:///CourseDataWarehouseDemo.db
Done.
Done.


IID,Name
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos
6,Scott M Brenner
7,Kevin C. Cassidy
8,Bruce Bradford
9,Milo W. Peck
10,Stephen E. Yost


In [25]:
%%sql

    SELECT COUNT(*)
    FROM db1.OFFERINGS_COURSE
        LEFT JOIN db1.CATALOG_COURSE ON (OFFERINGS_COURSE.CatalogID = CATALOG_COURSE.CatalogID)
            INNER JOIN CATALOG_COURSE_WH ON (CATALOG_COURSE.CatalogID = CATALOG_COURSE_WH.CatalogID)
        

 * sqlite:///CourseDataWarehouseDemo.db
Done.


COUNT(*)
15243


In [15]:
%%sql

SELECT IID
FROM REG_COURSE_WH
LIMIT 10;

 * sqlite:///CourseDataWarehouseDemo.db
Done.


IID
None
None
None
None
None
None
None
None
None
None


Vacuum

In [28]:
%%sql

vacuum;

 * sqlite:///CourseDataWarehouseDemo.db
Done.


[]